In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/udipt/Desktop/Websoft/Assignment/DataScienceCaseStudy/dataSample.csv')

# Preprocessing

In [3]:
df=pd.get_dummies(df)
df=df.rename(columns={'Unnamed: 0':'sl_number'})

In [4]:
df=df.drop(columns=['event_label_activate','warranty_service_flag_n'],axis=1)

In [5]:
df['log_reading_1'].fillna(df['log_reading_1'].median(),inplace=True)
df['log_reading_3'].fillna(df['log_reading_3'].median(),inplace=True)

# Train_test

In [6]:
x=df.drop(columns='event_label_break',axis=1)
y=df['event_label_break']

In [7]:
from sklearn.model_selection import train_test_split
x_tr,x_te,y_tr,y_te=train_test_split(x,y,stratify=y,test_size=.27,random_state=10)
print(x_tr.shape,x_te.shape,y_tr.shape,y_te.shape)
print(f'Training set has {y_tr.mean()*100:.2f}% positive points and test sets {y_te.mean()*100:.2f}% of positive points')

(11798, 13) (4364, 13) (11798,) (4364,)
Training set has 34.15% positive points and test sets 34.14% of positive points


# important features

In [1]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
rf=RandomForestClassifier(n_jobs=-1,random_state=10)
rf.fit(x_tr,y_tr)

C:\Users\udipt\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [10]:
rf.feature_importances_

array([0.2191413 , 0.10362834, 0.49376112, 0.08141107, 0.01848537,
       0.01782606, 0.03974661, 0.00563289, 0.00572136, 0.00524521,
       0.00488334, 0.00236348, 0.00215384])

In [11]:
print(f'Random forest predicts {rf.predict(x_te).mean()*100:.2f}% points as positive')

Random forest predicts 29.79% points as positive


In [12]:
x_te.columns

Index(['sl_number', 'ID', 'event_timestamp', 'log_timestamp', 'log_reading_1',
       'log_reading_3', 'log_reading_4', 'Type_TypeA', 'Type_TypeB',
       'log_reading_2_a', 'log_reading_2_b', 'log_reading_2_c',
       'warranty_service_flag_y'],
      dtype='object')

In [13]:
xi=x.drop(columns=['log_reading_1','log_reading_3','Type_TypeA', 'Type_TypeB',
       'log_reading_2_a', 'log_reading_2_b', 'log_reading_2_c',
       'warranty_service_flag_y'])
xi.head()

,sl_number,ID,event_timestamp,log_timestamp,log_reading_4
0,0,716013,1432329808,1432084088,7
1,1,716013,1539877380,1432084088,7
2,2,716013,1539879425,1432084088,7
3,3,716013,1539956727,1432084088,7
4,4,716013,1551462952,1432084088,7


# Train_test

In [14]:
xi_tr,xi_te,yi_tr,yi_te=train_test_split(xi,y,test_size=.27,stratify=y,random_state=10)
print(xi_tr.shape,xi_te.shape,yi_tr.shape,yi_te.shape)
print(f'Training set has {yi_tr.mean()*100:.2f}% positive points and test sets {yi_te.mean()*100:.2f}% of positive points')

(11798, 5) (4364, 5) (11798,) (4364,)
Training set has 34.15% positive points and test sets 34.14% of positive points


# XGboost

In [15]:
from xgboost import XGBClassifier

C:\Users\udipt\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  RuntimeWarning,


In [16]:
xg=XGBClassifier(base_score=.34,n_jobs=1,random_state=10)
xg.fit(xi_tr,yi_tr)

XGBClassifier(base_score=0.34, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=1, num_parallel_tree=1,
              objective='binary:logistic', random_state=10, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
print(f'xgboost predicts {xg.predict(xi_te).mean()*100:.2f}% points as positive')

xgboost predicts 37.88% points as positive


In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
pg={'n_estimators':range(470,500,10)}
xg_cv=GridSearchCV(xg,pg,cv=5,n_jobs=-1)
xg_cv.fit(xi_tr,yi_tr)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.34, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=1,
                                     num_parallel_tree=1,
                                     objective='binary:logistic',
                                     random_state=10, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, su

In [20]:
xg_cv.best_params_

{'n_estimators': 480}

In [21]:
xg_b=xg_cv.best_estimator_

In [22]:
print(f'xgboost predicts {xg_b.predict(xi_te).mean()*100:.2f}% points as positive')

xgboost predicts 35.24% points as positive


# Lift

In [23]:
xg_b.predict_proba(xi_te)

array([[7.1011758e-01, 2.8988242e-01],
       [9.9951696e-01, 4.8303505e-04],
       [1.9523227e-01, 8.0476773e-01],
       ...,
       [3.3781618e-01, 6.6218382e-01],
       [7.3783684e-01, 2.6216316e-01],
       [9.9818045e-01, 1.8195388e-03]], dtype=float32)

In [24]:
predicted_prob=[]
for i in xg_b.predict_proba(xi_te):
    predicted_prob.append(i[1])

In [25]:
df_y=pd.DataFrame(predicted_prob)
df_y.head()

,0
0,0.289882
1,0.000483
2,0.804768
3,0.118720
4,0.015068


In [26]:
test_set=yi_te.reset_index()
test_set.head()

,index,event_label_break
0,13877,1
1,6815,1
2,9851,0
3,16012,1
4,130,0


In [27]:
test_set=test_set.drop(columns='index',axis=1)

In [28]:
df_y['actual_values']=test_set
df_y=df_y.rename(columns={0:'predicted_prob'})

In [29]:
df_y=df_y.sort_values('predicted_prob',ascending=False)

In [30]:
df_y['decile_rank']=pd.qcut(df_y['predicted_prob'],10,labels=False)
df_y.head()

,predicted_prob,actual_values,decile_rank
1866,0.998824,1,9
278,0.998323,1,9
1420,0.998321,1,9
2388,0.998009,1,9
634,0.997397,0,9


In [31]:
df_y.to_csv('C:/Users/udipt/Desktop/Websoft/Lift csv/y.csv')

In [32]:
df_x=pd.DataFrame(xi_te.reset_index())
df_x.to_csv('C:/Users/udipt/Desktop/Websoft/Lift csv/x.csv')

In [33]:
x=pd.read_csv('C:/Users/udipt/Desktop/Websoft/Lift csv/x.csv')
y=pd.read_csv('C:/Users/udipt/Desktop/Websoft/Lift csv/y.csv')

In [34]:
x.columns,y.columns

(Index(['Unnamed: 0', 'index', 'sl_number', 'ID', 'event_timestamp',
        'log_timestamp', 'log_reading_4'],
       dtype='object'),
 Index(['Unnamed: 0', 'predicted_prob', 'actual_values', 'decile_rank'], dtype='object'))

In [35]:
final_df=pd.merge(left=x,right=y,left_on='Unnamed: 0',right_on='Unnamed: 0')
final_df=final_df.sort_values('predicted_prob',ascending=False)
final_df.head()

,Unnamed: 0,index,sl_number,ID,event_timestamp,log_timestamp,log_reading_4,predicted_prob,actual_values,decile_rank
1866,1866,2577,2577,320491,1600747211,1432962455,8,0.998824,1,9
278,278,13849,13849,641089,1600747211,1437023295,4,0.998323,1,9
1420,1420,8617,8617,567347,1600747211,1435046916,10,0.998321,1,9
2388,2388,11633,11633,64247,1600747211,1432786110,8,0.998009,1,9
634,634,5779,5779,710785,1600735958,1433923795,4,0.997397,0,9


In [36]:
final_df.groupby('decile_rank')[final_df.columns].mean()

,Unnamed: 0,index,sl_number,ID,event_timestamp,log_timestamp,log_reading_4,predicted_prob,actual_values,decile_rank
decile_rank,,,,,,,,,,
0,2092.897025,9062.205950,9062.205950,555055.938215,1.414381e+09,1.435167e+09,5.503432,0.000681,0.009153,0.0
1,2216.449541,7828.949541,7828.949541,550987.711009,1.417107e+09,1.434735e+09,5.853211,0.002759,0.020642,1.0
2,2176.795872,7646.616972,7646.616972,544221.949541,1.416090e+09,1.434721e+09,6.082569,0.007608,0.025229,2.0
3,2167.496568,7884.645309,7884.645309,496149.100686,1.425715e+09,1.434818e+09,5.958810,0.033076,0.141876,3.0
4,2192.580275,8383.000000,8383.000000,461983.986239,1.465516e+09,1.435003e+09,6.011468,0.154652,0.486239,4.0
5,2187.549199,8159.661327,8159.661327,474926.665904,1.480647e+09,1.434877e+09,5.931350,0.332793,0.594966,5.0
6,2194.582569,7590.733945,7590.733945,479345.518349,1.488346e+09,1.434693e+09,6.332569,0.504039,0.566514,6.0
7,2226.013761,8024.247706,8024.247706,479588.233945,1.496007e+09,1.434839e+09,6.350917,0.644716,0.552752,7.0
8,2197.137300,8014.123570,8014.123570,499675.379863,1.497725e+09,1.434835e+09,6.691076,0.772493,0.491991,8.0


In [37]:
l=[]
for i in final_df.columns:
    if i!='Unnamed: 0' and i!='index' and i!='sl_number' and i!='ID':
        l.append(i)
final_df.groupby('decile_rank')[l].mean()

,event_timestamp,log_timestamp,log_reading_4,predicted_prob,actual_values,decile_rank
decile_rank,,,,,,
0,1.414381e+09,1.435167e+09,5.503432,0.000681,0.009153,0.0
1,1.417107e+09,1.434735e+09,5.853211,0.002759,0.020642,1.0
2,1.416090e+09,1.434721e+09,6.082569,0.007608,0.025229,2.0
3,1.425715e+09,1.434818e+09,5.958810,0.033076,0.141876,3.0
4,1.465516e+09,1.435003e+09,6.011468,0.154652,0.486239,4.0
5,1.480647e+09,1.434877e+09,5.931350,0.332793,0.594966,5.0
6,1.488346e+09,1.434693e+09,6.332569,0.504039,0.566514,6.0
7,1.496007e+09,1.434839e+09,6.350917,0.644716,0.552752,7.0
8,1.497725e+09,1.434835e+09,6.691076,0.772493,0.491991,8.0


In [38]:
top_decile=final_df[final_df['decile_rank']==9]

In [39]:
top_decile.head()

,Unnamed: 0,index,sl_number,ID,event_timestamp,log_timestamp,log_reading_4,predicted_prob,actual_values,decile_rank
1866,1866,2577,2577,320491,1600747211,1432962455,8,0.998824,1,9
278,278,13849,13849,641089,1600747211,1437023295,4,0.998323,1,9
1420,1420,8617,8617,567347,1600747211,1435046916,10,0.998321,1,9
2388,2388,11633,11633,64247,1600747211,1432786110,8,0.998009,1,9
634,634,5779,5779,710785,1600735958,1433923795,4,0.997397,0,9


In [40]:
lift=top_decile['predicted_prob'].mean()/final_df['predicted_prob'].mean()

In [41]:
print(f'Lift = {lift:.2f}')

Lift = 2.71


In [42]:
import pickle

In [43]:
with open('C:/Users/udipt/Desktop/Websoft/Notebooks/Pickle/xg_grid','wb') as pxg:
    pickle.dump(xg_b,pxg)